# Model Server Testing

Test model server via HTTP calls 

In [1]:
# nuclio: ignore
import nuclio

In [2]:
# nuclio: start-code

In [3]:
%nuclio config spec.image = "mlrun/mlrun"
%nuclio config kind = "job"

%nuclio: setting spec.image to 'mlrun/mlrun'
%nuclio: setting kind to 'job'


In [4]:
import os
import pandas as pd
import requests
import json
import numpy as np
import pickle

def model_server_tester(context, model_endpoint, model_name, data_size, data_path, num_tests):
    # Ensure correct data types
    model_endpoint = str(model_endpoint)
    model_name = str(model_name)
    data_size = int(str(data_size))
    data_path = str(data_path)
    num_tests = int(str(num_tests))
    
    # Load data
    df = pd.read_pickle(data_path)
    df.drop('title', axis=1, inplace=True)
    
    # Get subset of data that was trained on
    df = df.sample(data_size, random_state=1)
    
    # Get random sample
    df = df.sample(num_tests)
    
    # Get text and target
    X = df['abstract'].to_list()
    y = df['chemicals'].to_list()
    
    # Get predictions
    for text, target in zip(X, y):
        event_body = json.dumps({"instances": [text]})
        headers = {'Content-type': 'application/json'}
        response = requests.post(url=model_endpoint + f'/{model_name}/predict', data=event_body, headers=headers)
        pred = response.content
        
        # Print
        print("Text:\n", text)
        print("Targets:\n", target)
        print("Preds:\n", str(pred))
        print("\n")

In [5]:
# nuclio: end-code
# marks the end of a code section

In [6]:
from mlrun import mlconf
import os
from os import path

# Target location for storing pipeline artifacts
artifact_path = path.abspath('../jobs')
# MLRun DB path or API service URL
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

print(f'Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}')

Artifacts path: /User/nlp/components/jobs
MLRun DB path: http://mlrun-api:8080


In [7]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("model_server_tester")

# add metadata (for templates and reuse)
fn.spec.default_handler = "model_server_tester"
fn.spec.description = "model server tester for fastai nlp"
fn.metadata.categories = ["serving", "ml"]
fn.metadata.labels = {"author": "nschenone"}
fn.export("../yaml/model_server_tester.yaml")

> 2020-08-13 19:02:50,370 [info] function spec saved to path: ../yaml/model_server_tester.yaml
